In [ ]:
# --- EVALUATION SCRIPT (DEĞERLENDİRME KODU) ---
# Bu kod, eğitilen modeli test verisi üzerinde dener.

# 1. Gerekli Kütüphaneler
!pip install -q torch transformers peft datasets

import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
from datasets import load_dataset

# 2. AYARLAR
# Base model (Orjinal hali)
base_model_id = "Qwen/Qwen2.5-Coder-1.5B-Instruct"

# Senin eğittiğin model (Hugging Face'deki ismi)
# BURAYI DÜZENLE: Kendi kullanıcı adını yaz
adapter_model_id = "caglademir/Qwen2.5-Coder-1.5B-LoRA-DEEP"

print(f"Test edilecek model: {adapter_model_id}")

# 3. MODELİ YÜKLE
# Önce base modeli yüklüyoruz
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

# Sonra senin eğittiğin LoRA adaptörünü üzerine ekliyoruz
model = PeftModel.from_pretrained(base_model, adapter_model_id)
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Modeli değerlendirme moduna al
model.eval()

# 4. TEST VERİSİNİ HAZIRLA
print("Test verisi yükleniyor...")
# Dokümana göre test split kullanılmalı
try:
    dataset = load_dataset("Naholav/CodeGen-Deep-5K", split="test")
except:
    # Eğer test split yoksa train'den küçük bir parça al
    print("Test split bulunamadı, Train setinden örnek alınıyor.")
    dataset = load_dataset("Naholav/CodeGen-Deep-5K", split="train").select(range(5))

# 5. ÖRNEK SORU ÇÖZME FONKSİYONU
def evaluate_model(index=0):
    input_text = dataset[index]['input']
    ground_truth = dataset[index]['solution']

    messages = [
        {"role": "system", "content": "You are an expert Python programmer."},
        {"role": "user", "content": input_text}
    ]

    prompt = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    print(f"\n--- SORU {index+1} ---")
    print(input_text[:200] + "...")

    # Model cevap üretiyor
    with torch.no_grad():
        outputs = model.generate(inputs.input_ids, max_new_tokens=256)

    response = tokenizer.decode(outputs[0], skip_special_tokens=True)

    # Cevabı temizle (Sadece üretilen kısmı al)
    response_code = response.split(prompt)[-1] if prompt in response else response

    print("\n--- MODELİN CEVABI ---")
    print(response_code)
    print("\n--- GERÇEK CEVAP (SOLUTION) ---")
    print(ground_truth[:300] + "...")

# 6. TESTİ ÇALIŞTIR
print("Değerlendirme Başlıyor...")
evaluate_model(0)
evaluate_model(1)